In [1]:
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import warnings
warnings.filterwarnings('ignore')

from enum import Enum
from copy import copy
from scipy.stats import norm
from scipy.cluster import hierarchy as sch
import pandas as pd
import numpy as np

import yfinance as yf

import sys, os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))


from _utils.strategies.fast_mean_reversion import *

DATA_DIR = "../_databases"

DEFAULT_DATE_FORMAT = "%Y-%m-%d"


In [2]:
#%% GET DATA
# # =================

def get_data_dict_with_hourly_adjusted(instrument_list: list):

    all_data = dict(
        [
            (instrument_code, pd.read_csv((DATA_DIR + '/' + f'{instrument_code}' + '.csv'),index_col='index'))
            for instrument_code in instrument_list
        ]
    )
    
    adjusted_prices_hourly = dict(
        [
            (instrument_code, pd.read_csv((DATA_DIR + '/' + f'{instrument_code}' + '_hourly.csv'),index_col='index'))
            for instrument_code in instrument_list
        ]
    )

    adjusted_prices = dict(
        [
            (instrument_code, data_for_instrument.adjusted)
            for instrument_code, data_for_instrument in all_data.items()
        ]   
    )
    
    current_prices = dict(
        [
            (instrument_code, data_for_instrument.underlying)
            for instrument_code, data_for_instrument in all_data.items()
        ]
    )
    
    carry_data = dict(
        [
            (instrument_code, pd.read_csv((DATA_DIR + '/' + f'{instrument_code}' + '_carry.csv'),index_col='index'))
            for instrument_code in instrument_list
        ]   
    )

    return adjusted_prices_hourly, adjusted_prices, current_prices, carry_data




def get_fx_data_dict(instrument_list: list):

    all_data = dict(
        [
            (instrument_code, pd.read_csv((DATA_DIR + '/' + f'{instrument_code}' + '.csv'),index_col='index'))
            for instrument_code in instrument_list
        ]
    )

    price = dict(
        [
            (instrument_code, data_for_instrument.price)
            for instrument_code, data_for_instrument in all_data.items()
        ]   
    )


    return price

In [3]:
# função auxiliar
def align_dict_to_index(data_dict, target_index):
    for inst in data_dict.keys():
        df = data_dict[inst]

        data_dict[inst] = df.reindex(target_index).ffill().bfill()

    return data_dict




In [32]:
adjusted_prices_hourly_dict, adjusted_prices_dict, current_prices_dict, carry_prices_dict = get_data_dict_with_hourly_adjusted(["sp500"])


fx_data_dict = get_fx_data_dict(['eur_fx'])
multipliers = dict(sp500=5, eurostx=10, us10=1000, us2=2000)
risk_target_tau = 0.2

fx_series_dict = create_fx_series_given_adjusted_prices_dict(adjusted_prices_dict,dict(sp500=1, eurostx=fx_data_dict['eur_fx'], us10=1, us2=1))

capital = 1000000

idm = 1.4
instrument_weights = dict(sp500=0.5, us10=0.5, us2=0.3333)

commission_per_contract_dict = dict(sp500=0.6, us10=1.51, us2=1.51)
bid_ask_spread_dict = dict(sp500=0.25, us10=1 / 64, us2=(1 / 8) * (1 / 32))
tick_size_dict = dict(sp500=0.25, us10=1 / 64, us2=(1 / 8) * (1 / 32))

std_dev_dict = calculate_variable_standard_deviation_for_risk_targeting_from_dict(
    adjusted_prices=adjusted_prices_dict,
    current_prices=current_prices_dict,
)

In [42]:
average_position_contracts_dict = (
        calculate_position_series_given_variable_risk_for_dict(
            capital=capital,
            risk_target_tau=risk_target_tau,
            idm=idm,
            weights=instrument_weights,
            std_dev_dict=std_dev_dict,
            fx_series_dict=fx_series_dict,
            multipliers=multipliers,
        )
    )

# average_position_contracts_dict = apply_buffering_to_position_dict(
#     position_contracts_dict=average_position_contracts_dict,
#     average_position_contracts_dict=average_position_contracts_dict,
# )

# perc_returns_dict = generate_pandl_across_instruments_for_hourly_data(
#         adjusted_prices_hourly_dict=adjusted_prices_hourly_dict,
#         adjusted_prices_daily_dict=adjusted_prices_dict,
#         fx_series_dict=fx_series_dict,
#         multipliers=multipliers,
#         commission_per_contract_dict=commission_per_contract_dict,
#         average_position_contracts_dict=average_position_contracts_dict,
#         std_dev_dict=std_dev_dict,
#         current_prices_daily_dict=current_prices_dict,
#         capital=capital,
#         trade_calculation_function=required_orders_for_mr_system,
#         tick_size_dict=tick_size_dict,
#         bid_ask_spread_dict=bid_ask_spread_dict,
#     )

In [43]:
adjusted_daily_prices = adjusted_prices_dict['sp500']
adjusted_hourly_prices = adjusted_prices_hourly_dict['sp500']
current_daily_prices = current_prices_dict['sp500']
daily_stdev = std_dev_dict['sp500']

dates = list(adjusted_hourly_prices.index)
dt = dates[1]


average_position_daily = average_position_contracts_dict['sp500']
daily_eq_hourly = calculate_equilibrium(
        adjusted_daily_prices=adjusted_daily_prices,
        adjusted_hourly_prices=adjusted_hourly_prices,
    )

hourly_sigma = calculate_sigma_p(
        current_daily_prices=current_daily_prices,
        daily_stdev=daily_stdev,
        adjusted_hourly_prices=adjusted_hourly_prices,
    )

price = float(get_row_of_series_before_date(adjusted_hourly_prices, dt))
eq = get_row_of_series_before_date(daily_eq_hourly, dt)
avg = get_row_of_series_before_date(average_position_daily, dt)
sigma = get_row_of_series_before_date(hourly_sigma, dt)

forecast = mr_forecast_unclipped(
        current_equilibrium=eq,
        current_hourly_stdev_price=sigma,
        current_price=price,
    )



target_position = optimal_position_given_unclipped_forecast(
        current_forecast=forecast,
        current_average_position=avg,
    )


forecast = mr_forecast_unclipped(
        current_equilibrium=eq,
        current_hourly_stdev_price=sigma,
        current_price=price,
    )

orders = calculate_orders_given_forecast_and_positions(
        current_average_position=avg,
        current_forecast=forecast,
        current_equilibrium=eq,
        current_hourly_stdev_price=sigma,
        current_position=0,
        tick_size=tick_size_dict['sp500'],
    )

In [38]:

trade = fill_list_of_orders(orders, dt, price, bid_ask_spread_dict['sp500'])
trade

Trade(qty=0, fill_date='2013-10-18 02:00:00', current_price=1563.645833333333)

In [7]:
trades = generate_list_of_mr_trades_for_instrument(
        adjusted_daily_prices=adjusted_daily_prices.tail(20),
        current_daily_prices=current_daily_prices.tail(20),
        adjusted_hourly_prices=adjusted_hourly_prices.tail(20),
        daily_stdev=daily_stdev.tail(20),
        average_position_daily=average_position_daily.tail(20),
        tick_size=tick_size_dict['sp500'],
        bid_ask_spread=bid_ask_spread_dict['sp500'],
        instrument_code=['sp500'],
        trade_calculation_function=required_orders_for_mr_system,
    )

In [8]:
trades

[]

In [ ]:
list_of_trades = trades
qty_list = [t.qty for t in list_of_trades]
date_list = [t.fill_date for t in list_of_trades]
price_list = [t.current_price for t in list_of_trades]

qty_series = pd.Series(qty_list, index=date_list)
price_series = pd.Series(price_list, index=date_list)
position_series = qty_series.cumsum()

AttributeError: 'ListOfOrders' object has no attribute 'qty'

In [ ]:
# --- Plot retorno acumulado
fig = go.Figure()

fig.add_trace(
    go.Scatter(
        x=perc_returns_dict.index,
        y=perc_returns_dict.cumsum()*100,
        name="Skew",
        line=dict(color="black"),
    )
)


fig.update_layout(
    title="Strategies for Future Contracts Portfolio",
    xaxis_title="Data",
    yaxis_title="Acumulated return (%)",
    template="plotly_white",
    showlegend=True,
    legend=dict(
        yanchor="bottom",
        y=0.8,
        xanchor="left",
        x=0.0),
    height=600,
    width=1000,
)

fig.show()

# --- Exibir estatísticas
print("Estatísticas de Desempenho")
print("Estatísticas Portfolio Skew Frequência Anual")
print(calculate_stats(perc_returns_dict, freq=YEAR))
